### Setting up the problem

In [3]:
from qiskit_nature.drivers import PySCFDriver

molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom=molecule)
qmolecule = driver.run()

## Preprocessing

#### Reducing the number of qubits in the operator:
  - Freezing the core
  - Using parity mapper
  - Note that no Z2Symmetries left there

In [4]:
from qiskit_nature.transformers import FreezeCoreTransformer
from qiskit_nature.mappers.second_quantization import ParityMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem


# Instantiate a FreezeCoreTransformer to freeze the core and remove orbitals which can be removed
FCT = FreezeCoreTransformer(freeze_core=True, remove_orbitals=[3,4])

# ElectronicStructureProblem object, transforms the original qmolecule
problem = ElectronicStructureProblem(driver=driver, q_molecule_transformers=[FCT])

# Get operator of the problem
main_op = problem.second_q_ops()[0]

# ParityMapper with 2 qubit reduction. This converter has the biggest advantage in this problem
converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True) 

# reduced, actual number of particles
num_particles = (problem.molecule_data_transformed.num_alpha,
             problem.molecule_data_transformed.num_beta) 

# reduced num of orbitals
num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals 
# produce operator
qubit_op = converter.convert(main_op, num_particles=num_particles) 

# Printed to see if there's any
print(converter.z2symmetries) 


Z2 symmetries:
Symmetries:
Single-Qubit Pauli X:
Cliffords:
Qubit index:
[]
Tapering values:
  - Possible values: []


### Build the ansatz

#### TwoLocal, with parameters:
  - number of qubits (that's 4 here)
  - rotation blocks, for each axis, but with minimized parameters
  - entanglement block is cnot (has no param)
  - linear entanglement since it has the least number of cnots in it with full connection
  - don't skip the final rotations, they don't count in the score!

In [5]:
from qiskit.circuit.library import TwoLocal

ansatz = TwoLocal(num_qubits=qubit_op.num_qubits,
                  rotation_blocks=['z','x','ry'], # only 1 paramterized gate
                  entanglement_blocks='cx',       
                  reps=1,                         # one is enough
                  entanglement='linear',          # minimal cnots with full connection
                  skip_final_rotation_layer=False)# rotations do the work, and they doesn't count in the score

print(ansatz)

     ┌───┐┌───┐┌──────────┐     ┌───┐┌───┐┌──────────┐                        
q_0: ┤ Z ├┤ X ├┤ RY(θ[0]) ├──■──┤ Z ├┤ X ├┤ RY(θ[4]) ├────────────────────────
     ├───┤├───┤├──────────┤┌─┴─┐└───┘├───┤└──┬───┬───┘┌──────────┐            
q_1: ┤ Z ├┤ X ├┤ RY(θ[1]) ├┤ X ├──■──┤ Z ├───┤ X ├────┤ RY(θ[5]) ├────────────
     ├───┤├───┤├──────────┤└───┘┌─┴─┐└───┘   ├───┤    └──┬───┬───┘┌──────────┐
q_2: ┤ Z ├┤ X ├┤ RY(θ[2]) ├─────┤ X ├──■─────┤ Z ├───────┤ X ├────┤ RY(θ[6]) ├
     ├───┤├───┤├──────────┤     └───┘┌─┴─┐   ├───┤       ├───┤    ├──────────┤
q_3: ┤ Z ├┤ X ├┤ RY(θ[3]) ├──────────┤ X ├───┤ Z ├───────┤ X ├────┤ RY(θ[7]) ├
     └───┘└───┘└──────────┘          └───┘   └───┘       └───┘    └──────────┘


### Building the algorithm

#### Given:
  - initial_point, that was given by the excercise
  - init_state was also given
  - callback is just the callback from the tutorial
  
#### VQE:
  - Optimizer: SLSQP, with not too many params
  - Ansatz: the TwoLocal above
  

In [1]:
from qiskit_nature.circuit.library import HartreeFock
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms import VQE
from IPython.display import display, clear_output
from qiskit import Aer

# Given
initial_point = [0.01] * ansatz.num_parameters
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)

# Callback to trace the optimizations process
counts = []
values = []
params = []
deviation = []
def callback(eval_count, parameters, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    params.append(parameters)
    deviation.append(std)


# Build the VQE with SLSQP with high maxiter number, since it stops before reaching it
# And SLSQP is a great QP solver to do this, with moderate number of params
algorithm = VQE(ansatz,
                optimizer=SLSQP(maxiter=3000),
                quantum_instance=Aer.get_backend('statevector_simulator'),
                callback=callback,
                initial_point=initial_point)

# Compute the result
result = algorithm.compute_minimum_eigenvalue(qubit_op)


Traceback (most recent call last):
  File "<ipython-input-1-439d647f8e96>", line 7, in <module>
    initial_point = [0.01] * ansatz.num_parameters
NameError: name 'ansatz' is not defined

Use %tb to get the full traceback.


### Getting the exact solution

#### The same way as in the excercise, to compare with the result

Note: this is not necessary step, even for bigger molecules it's just inefficient.

In [ ]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])

# Exact solution to reach
print("Exact electronic energy", exact_energy) 

In [ ]:
from qc_grader import grade_ex5
freeze_core = True 
grade_ex5(ansatz,qubit_op,result,freeze_core)

In [ ]:
from qc_grader import submit_ex5
submit_ex5(ansatz,qubit_op,result,freeze_core)

### Author: András Czégel

  - Score: 3
  - Chemical accuracy (error) : 2.339090 mHa 
  - Number of parameters: 8